# Rewrite-Retrieve-Read

**Rewrite-Retrieve-Read** is a method proposed in the paper [Query Rewriting for Retrieval-Augmented Large Language Models](https://arxiv.org/pdf/2305.14283.pdf)

> Because the original query can not be always optimal to retrieve for the LLM, especially in the real world... we first prompt an LLM to rewrite the queries, then conduct retrieval-augmented reading

We show how you can easily do that with LangChain Expression Language

## Baseline

Baseline RAG (**Retrieve-and-read**) can be done like the following:

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
template = """Answer the users question based only on the following context:

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0)

search = DuckDuckGoSearchAPIWrapper()


def retriever(query):
    return search.run(query)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
simple_query = "what is langchain?"

In [ ]:
chain.invoke(simple_query)

While this is fine for well formatted queries, it can break down for more complicated queries

In [ ]:
distracted_query = "man that sam bankman fried trial was crazy! what is langchain?"

In [ ]:
chain.invoke(distracted_query)

This is because the retriever does a bad job with these "distracted" queries

In [ ]:
retriever(distracted_query)

## Rewrite-Retrieve-Read Implementation

The main part is a rewriter to rewrite the search query

In [ ]:
template = """Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain import hub

rewrite_prompt = hub.pull("langchain-ai/rewrite")

In [ ]:
print(rewrite_prompt.template)

In [ ]:
# Parser to remove the `**`


def _parse(text):
    return text.strip('"').strip("**")

In [ ]:
rewriter = rewrite_prompt | ChatOpenAI(temperature=0) | StrOutputParser() | _parse

In [ ]:
rewriter.invoke({"x": distracted_query})

In [ ]:
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rewrite_retrieve_read_chain.invoke(distracted_query)